In [5]:
import os
import pandas as pd
import google.generativeai as genai
from google.colab import drive, files
from google.colab import userdata
!pip install PyPDF2 # Ensure PyPDF2 is installed
import PyPDF2
import io
import re
from datetime import datetime
import json















In [6]:
# --- Configuration for Standardization ---
# You can expand these mappings as needed
TEST_NAME_MAPPING = {
    # Hematology
    re.compile(r'haemoglobin', re.IGNORECASE): 'Haemoglobin',
    re.compile(r'hgb', re.IGNORECASE): 'Haemoglobin',
    re.compile(r'total leucocyte count', re.IGNORECASE): 'Total Leucocyte Count',
    re.compile(r'tlc', re.IGNORECASE): 'Total Leucocyte Count',
    re.compile(r'white blood cell count', re.IGNORECASE): 'Total Leucocyte Count',
    re.compile(r'wbc', re.IGNORECASE): 'Total Leucocyte Count',
    re.compile(r'platelet count', re.IGNORECASE): 'Platelet Count',
    re.compile(r'rbc count', re.IGNORECASE): 'Red Blood Cell Count',
    re.compile(r'red cell count', re.IGNORECASE): 'Red Blood Cell Count',
    re.compile(r'hematocrit', re.IGNORECASE): 'Hematocrit',
    re.compile(r'hct', re.IGNORECASE): 'Hematocrit',
    re.compile(r'packed cell volume', re.IGNORECASE): 'Hematocrit',
    re.compile(r'pcv', re.IGNORECASE): 'Hematocrit',
    re.compile(r'mcv', re.IGNORECASE): 'Mean Corpuscular Volume (MCV)',
    re.compile(r'mch', re.IGNORECASE): 'Mean Corpuscular Haemoglobin (MCH)',
    re.compile(r'mchc', re.IGNORECASE): 'Mean Corpuscular Haemoglobin Concentration (MCHC)',
    re.compile(r'rdw-cv', re.IGNORECASE): 'Red Cell Distribution Width - CV (RDW-CV)',
    re.compile(r'rdw-sd', re.IGNORECASE): 'Red Cell Distribution Width - SD (RDW-SD)',
    re.compile(r'neutrophils?', re.IGNORECASE): 'Neutrophils', # Matches Neutrophil and Neutrophils
    re.compile(r'lymphocytes?', re.IGNORECASE): 'Lymphocytes',
    re.compile(r'monocytes?', re.IGNORECASE): 'Monocytes',
    re.compile(r'eosinophils?', re.IGNORECASE): 'Eosinophils',
    re.compile(r'basophils?', re.IGNORECASE): 'Basophils',
    re.compile(r'esr', re.IGNORECASE): 'Erythrocyte Sedimentation Rate (ESR)',
    # Liver Function
    re.compile(r'total bilirubin', re.IGNORECASE): 'Bilirubin - Total',
    re.compile(r'direct bilirubin', re.IGNORECASE): 'Bilirubin - Direct',
    re.compile(r'indirect bilirubin', re.IGNORECASE): 'Bilirubin - Indirect',
    re.compile(r'sgpt', re.IGNORECASE): 'Alanine Aminotransferase (SGPT/ALT)',
    re.compile(r'alt', re.IGNORECASE): 'Alanine Aminotransferase (SGPT/ALT)',
    re.compile(r'sgot', re.IGNORECASE): 'Aspartate Aminotransferase (SGOT/AST)',
    re.compile(r'ast', re.IGNORECASE): 'Aspartate Aminotransferase (SGOT/AST)',
    re.compile(r'alkaline phosphatase', re.IGNORECASE): 'Alkaline Phosphatase (ALP)',
    re.compile(r'alp', re.IGNORECASE): 'Alkaline Phosphatase (ALP)',
    re.compile(r'total protein', re.IGNORECASE): 'Protein - Total',
    re.compile(r'albumin', re.IGNORECASE): 'Albumin',
    re.compile(r'globulin', re.IGNORECASE): 'Globulin',
    re.compile(r'a/g ratio', re.IGNORECASE): 'Albumin/Globulin Ratio (A/G Ratio)',
    # Kidney Function
    re.compile(r'urea', re.IGNORECASE): 'Urea',
    re.compile(r'blood urea nitrogen', re.IGNORECASE): 'Blood Urea Nitrogen (BUN)',
    re.compile(r'bun', re.IGNORECASE): 'Blood Urea Nitrogen (BUN)',
    re.compile(r'creatinine', re.IGNORECASE): 'Creatinine',
    re.compile(r'uric acid', re.IGNORECASE): 'Uric Acid',
    # Lipids
    re.compile(r'total cholesterol', re.IGNORECASE): 'Cholesterol - Total',
    re.compile(r'triglycerides', re.IGNORECASE): 'Triglycerides',
    re.compile(r'hdl cholesterol', re.IGNORECASE): 'HDL Cholesterol',
    re.compile(r'ldl cholesterol', re.IGNORECASE): 'LDL Cholesterol',
    re.compile(r'vldl cholesterol', re.IGNORECASE): 'VLDL Cholesterol',
    re.compile(r'cholesterol/hdl ratio', re.IGNORECASE): 'Total Cholesterol/HDL Ratio',
    # Diabetes
    re.compile(r'glucose fasting', re.IGNORECASE): 'Glucose - Fasting',
    re.compile(r'fbs', re.IGNORECASE): 'Glucose - Fasting',
    re.compile(r'glucose random', re.IGNORECASE): 'Glucose - Random',
    re.compile(r'rbs', re.IGNORECASE): 'Glucose - Random',
    re.compile(r'hba1c', re.IGNORECASE): 'Glycated Haemoglobin (HbA1c)',
    # Thyroid
    re.compile(r'tsh', re.IGNORECASE): 'Thyroid Stimulating Hormone (TSH)',
    re.compile(r'total t3', re.IGNORECASE): 'Total T3',
    re.compile(r'total t4', re.IGNORECASE): 'Total T4',
    re.compile(r'free t3', re.IGNORECASE): 'Free T3 (FT3)',
    re.compile(r'ft3', re.IGNORECASE): 'Free T3 (FT3)',
    re.compile(r'free t4', re.IGNORECASE): 'Free T4 (FT4)',
    re.compile(r'ft4', re.IGNORECASE): 'Free T4 (FT4)',
}

UNIT_MAPPING = {
    re.compile(r'gm/dl', re.IGNORECASE): 'g/dL',
    re.compile(r'g/l', re.IGNORECASE): 'g/L',
    re.compile(r'cells/cumm', re.IGNORECASE): 'cells/µL',
    re.compile(r'/cumm', re.IGNORECASE): '/µL',
    re.compile(r'thou/cumm', re.IGNORECASE): 'x10³/µL',
    re.compile(r'10\^3/ul', re.IGNORECASE): 'x10³/µL',
    re.compile(r'x10\^3/μl', re.IGNORECASE): 'x10³/µL',
    re.compile(r'mill/cumm', re.IGNORECASE): 'x10⁶/µL',
    re.compile(r'10\^6/ul', re.IGNORECASE): 'x10⁶/µL',
    re.compile(r'x10\^6/μl', re.IGNORECASE): 'x10⁶/µL',
    re.compile(r'mg/dl', re.IGNORECASE): 'mg/dL',
    re.compile(r'iu/l', re.IGNORECASE): 'IU/L',
    re.compile(r'u/l', re.IGNORECASE): 'U/L', # Common variation for IU/L
    re.compile(r'µiu/ml', re.IGNORECASE): 'µIU/mL', # For TSH etc.
    re.compile(r'ng/dl', re.IGNORECASE): 'ng/dL', # For some hormones
    re.compile(r'pg/ml', re.IGNORECASE): 'pg/mL', # For FT3 etc.
    re.compile(r'%', re.IGNORECASE): '%', # Percentage
    re.compile(r'fl', re.IGNORECASE): 'fL', # Femtoliters for MCV
    re.compile(r'pg', re.IGNORECASE): 'pg', # Picograms for MCH
    re.compile(r'mm/hr', re.IGNORECASE): 'mm/hr', # For ESR
}

STATUS_MAPPING = {
    re.compile(r'low', re.IGNORECASE): 'Low',
    re.compile(r'normal', re.IGNORECASE): 'Normal',
    re.compile(r'high', re.IGNORECASE): 'High',
    re.compile(r'critical', re.IGNORECASE): 'Critical',
    re.compile(r'n/a', re.IGNORECASE): 'N/A',
    re.compile(r'not applicable', re.IGNORECASE): 'N/A',
    re.compile(r'within normal limits', re.IGNORECASE): 'Normal',
    re.compile(r'within reference range', re.IGNORECASE): 'Normal',
    re.compile(r'near optimal', re.IGNORECASE): 'Near Optimal', # From user's example output
    re.compile(r'desirable', re.IGNORECASE): 'Desirable',
    re.compile(r'borderline high', re.IGNORECASE): 'Borderline High',
}

In [7]:
def standardize_value(value, mapping_dict, default_case='title'):
    """Applies mapping and case standardization to a value."""
    if not isinstance(value, str):
        return value # Return as is if not a string

    original_value = value.strip()
    standardized_value = original_value

    for pattern, standard_form in mapping_dict.items():
        if pattern.search(original_value):
            standardized_value = standard_form
            break # Apply first match

    # Apply case standardization if no mapping was applied or if mapped value needs it
    if default_case == 'title':
        return standardized_value.title() if standardized_value else standardized_value
    elif default_case == 'original': # Used for units where case matters (e.g. g/dL)
        return standardized_value
    elif default_case == 'lower':
        return standardized_value.lower() if standardized_value else standardized_value
    return standardized_value

In [8]:
# Mount Google Drive
try:
    drive.mount('/content/drive', force_remount=True)
except Exception as e:
    print(f"Error mounting drive: {e}")

# Configure Gemini API
try:
    GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
    genai.configure(api_key=GEMINI_API_KEY)
    model = genai.GenerativeModel('gemini-1.5-flash')
    print("Gemini API configured successfully.")
except Exception as e:
    print(f"Please add your Gemini API key to Colab secrets with name 'GEMINI_API_KEY'. Error: {e}")
    model = None

def extract_text_from_pdf(file_path):
    """Extract text from PDF file"""
    try:
        with open(file_path, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            text = ""
            for page in pdf_reader.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text
        if not text.strip():
            print(f"Warning: No text extracted from {os.path.basename(file_path)}. PDF might be image-based or empty.")
        return text
    except Exception as e:
        print(f"Error reading PDF {file_path}: {str(e)}")
        return None


Mounted at /content/drive
Gemini API configured successfully.


In [9]:
def analyze_medical_report(text):
    """Use API to analyze medical report and extract structured data"""
    if not model:
        print("Gemini model not initialized. Cannot analyze report.")
        return None
    if not text or not text.strip():
        print("No text provided to analyze.")
        return None

    # Enhanced prompt for better date and patient info extraction
    prompt = """
    Analyze this medical test report. Extract all patient information and test results.
    The patient's full name is critical. Also extract Patient ID, Age, and Gender.
    The report date or collection date is also critical. Ensure date is in<y_bin_358>-MM-DD format if possible.

    For each test parameter, extract:
    - Test Name (e.g., "Haemoglobin", "Total Leucocyte Count")
    - Result (numerical value or finding like "Detected", "Not Detected")
    - Unit of measurement (e.g., "g/dL", "cells/µL")
    - Reference Range (e.g., "13.0 - 17.0")
    - Status (interpret as "Low", "Normal", "High", "Critical", or "N/A" if not applicable or clearly stated)
    - Category (e.g., "Haematology", "Liver Function Test")

    Return the data in this exact JSON format:
    {
        "patient_info": {
            "name": "Full Patient Name",
            "age": "Age (e.g., 35 years, 35 Y)",
            "gender": "Gender (e.g., Male, Female)",
            "patient_id": "Patient ID or Registration No.",
            "date": "Test Date or Report Date (YYYY-MM-DD)"
        },
        "test_results": [
            {
                "test_name": "Name of the test",
                "result": "Numerical value or finding",
                "unit": "Unit of measurement",
                "reference_range": "Normal reference range",
                "status": "Low/Normal/High/Critical/N/A",
                "category": "Test category"
            }
        ],
        "abnormal_findings": [
            "List any general abnormal findings or summary remarks from the report, if present."
        ]
    }

    Medical Report Text:
    """ + text

    try:
        response = model.generate_content(prompt)
        response_text = response.text
        match = re.search(r'\{.*\}', response_text, re.DOTALL)
        if match:
            json_str = match.group(0)
            try:
                return json.loads(json_str)
            except json.JSONDecodeError as json_e:
                print(f"Error decoding JSON from response: {json_e}")
                print(f"Problematic JSON string (first 500 chars): {json_str[:500]}")
                return None
        else:
            print("Could not find valid JSON in API response.")
            print(f"API Response text (first 500 chars): {response_text[:500]}")
            return None
    except Exception as e:
        print(f"Error analyzing report with Gemini: {str(e)}")
        return None

In [10]:
def process_drive_folder(folder_path):
    """Process all PDF files in the specified Google Drive folder"""
    all_results_from_ai = []
    if not os.path.exists(folder_path):
        print(f"Folder not found: {folder_path}")
        return None
    pdf_files = [f for f in os.listdir(folder_path) if f.lower().endswith('.pdf')]
    if not pdf_files:
        print(f"No PDF files found in the folder: {folder_path}")
        return None

    print(f"Found {len(pdf_files)} PDF files to process in {folder_path}...")
    for i, filename in enumerate(pdf_files, 1):
        print(f"\nProcessing file {i}/{len(pdf_files)}: {filename}")
        file_path = os.path.join(folder_path, filename)
        text = extract_text_from_pdf(file_path)
        if not text:
            print(f"Could not extract text from {filename}, skipping analysis.")
            continue
        analysis = analyze_medical_report(text)
        if not analysis:
            print(f"Could not analyze {filename}, skipping.")
            continue
        analysis['source_filename'] = filename # Add source filename for traceability
        all_results_from_ai.append(analysis)
        print(f"✓ Successfully processed and analyzed {filename}")
    return all_results_from_ai

In [11]:
def create_cleaned_single_csv(all_ai_results, output_filename='cleaned_medical_reports_analysis.csv'):
    """
    Creates a single, cleaned CSV file in long format from all AI analysis results.
    Includes both the original and standardized Test_Name.
    The DataFrame is sorted by Test_Category, then Test_Name.
    """
    if not all_ai_results:
        print("No AI results to process for CSV creation.")
        return pd.DataFrame()

    all_rows_for_csv = []

    for report_json in all_ai_results:
        patient_info = report_json.get('patient_info', {})
        source_file = report_json.get('source_filename', 'N/A')

        patient_id = patient_info.get('patient_id', 'N/A')
        age = patient_info.get('age', 'N/A')
        gender = patient_info.get('gender', 'N/A')
        test_date = patient_info.get('date', 'N/A')

        for test_result in report_json.get('test_results', []):
            raw_test_name = test_result.get('test_name', 'UnknownTest') # Get the original name
            raw_unit = test_result.get('unit', '')
            raw_status = test_result.get('status', '')

            std_test_name = standardize_value(raw_test_name, TEST_NAME_MAPPING, default_case='title')
            std_unit = standardize_value(raw_unit, UNIT_MAPPING, default_case='original')
            std_status = standardize_value(raw_status, STATUS_MAPPING, default_case='title')

            row = {
                'Source_Filename': source_file,
                'Patient_ID': patient_id,
                'Age': age,
                'Gender': gender,
                'Test_Date': test_date,
                'Test_Category': test_result.get('category', 'N/A'),
                'Original_Test_Name': raw_test_name,  # Add the original test name here
                'Test_Name': std_test_name,
                'Result': test_result.get('result', ''),
                'Unit': std_unit,
                'Reference_Range': test_result.get('reference_range', ''),
                'Status': std_status,
                'Processed_Date': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            }
            all_rows_for_csv.append(row)

    if not all_rows_for_csv:
        print("No data rows were generated for the CSV.")
        return pd.DataFrame()

    df = pd.DataFrame(all_rows_for_csv)

    # Update column_order to include 'Original_Test_Name'
    column_order = [
        'Source_Filename', 'Patient_ID', 'Age', 'Gender', 'Test_Date',
        'Test_Category', 'Original_Test_Name', 'Test_Name', 'Result', 'Unit', 'Reference_Range', 'Status',
        'Processed_Date'
    ]
    final_columns = [col for col in column_order if col in df.columns]
    df = df[final_columns] # Reorder columns to the desired order

    # Sort the DataFrame
    sort_by_keys = []

    if 'Test_Name' in df.columns:
        sort_by_keys.append('Test_Name')

    if 'Test_Date' in df.columns: # For consistent ordering within Test_Name
        sort_by_keys.append('Test_Date')

    if 'Test_Category' in df.columns:
        sort_by_keys.append('Test_Category')


    if 'Patient_ID' in df.columns: # Further tie-breaker
        sort_by_keys.append('Patient_ID')

    if sort_by_keys: # Ensure there's at least one valid key to sort by
        df = df.sort_values(by=sort_by_keys).reset_index(drop=True)
    else:
        print("Warning: Could not sort DataFrame as necessary columns are missing.")


    try:
        df.to_csv(output_filename, index=False)
        print(f"\n✓ Cleaned single CSV file saved as: {output_filename}")
        print(f"  Total records (test entries): {len(df)}")
        if not df.empty:
            print(f"  Unique standardized test names found: {df['Test_Name'].nunique()}")
        return df
    except Exception as e:
        print(f"Error saving CSV file {output_filename}: {e}")
        return pd.DataFrame()




In [15]:
# --- Main execution ---
def main():
    """Main function to process medical reports and generate a single cleaned CSV."""
    if not model:
        print("Terminating main function as Gemini model is not initialized.")
        return

    FOLDER_PATH = input("Enter the path to your Google Drive folder containing PDF reports (e.g., /content/drive/MyDrive/Medical_Reports): ")

    print("\nStarting medical report analysis...")
    print("="*50)

    all_extracted_data_from_pdfs = process_drive_folder(FOLDER_PATH)

    if not all_extracted_data_from_pdfs:
        print("No data was successfully extracted from any PDF files. Exiting.")
        return

    print(f"\n✓ Successfully processed {len(all_extracted_data_from_pdfs)} PDF file(s) and extracted raw data structures.")

    print("\n--- Generating Single Cleaned CSV File ---")
    final_df = create_cleaned_single_csv(all_extracted_data_from_pdfs)

    print("\n" + "="*50)
    print("ANALYSIS COMPLETE!")
    print("="*50)

    if final_df.empty:
        print("No CSV file was generated or the generated file was empty.")
    else:
        print(f"A single cleaned CSV has been generated: '{'cleaned_medical_reports_analysis.csv'}'")
        print(f"  It contains {len(final_df)} test entries.")

    output_csv_name = 'cleaned_medical_reports_analysis.csv'
    if not final_df.empty and os.path.exists(output_csv_name):
        download_choice = input(f"\nDo you want to download the '{output_csv_name}' file? (y/n): ").lower()
        if download_choice == 'y':
            files.download(output_csv_name)
            print(f"Downloading {output_csv_name}...")
    elif not final_df.empty:
        print(f"Warning: CSV file {output_csv_name} was processed but not found on disk for download.")

    print("\nProcessing finished. The CSV file is saved in the Colab environment.")
    print(f"You can find it in the directory: {os.getcwd()}")

if __name__ == "__main__":
    if model:
        main()
    else:
        print("Main function not executed because Gemini model initialization failed. Please check your API key setup.")

print("\nScript setup complete.")
print("If the Gemini API is configured, you can run the `main()` function cell to start processing.")
print("Instructions:")
print("1. Ensure your Gemini API key is added to Colab secrets (View -> Secrets -> Add new secret 'GEMINI_API_KEY').")
print("2. Upload your PDF medical reports to a folder in your Google Drive.")
print("3. When prompted by `main()`, enter the full path to that Google Drive folder.")
print("4. The script will process the PDFs and generate a single, cleaned CSV file named 'cleaned_medical_reports_analysis.csv'.")

Enter the path to your Google Drive folder containing PDF reports (e.g., /content/drive/MyDrive/Medical_Reports): /content/drive/MyDrive/Tests/Test Reports/Parmaansh

Starting medical report analysis...
Found 1 PDF files to process in /content/drive/MyDrive/Tests/Test Reports/Parmaansh...

Processing file 1/1: 30403605847.pdf
✓ Successfully processed and analyzed 30403605847.pdf

✓ Successfully processed 1 PDF file(s) and extracted raw data structures.

--- Generating Single Cleaned CSV File ---

✓ Cleaned single CSV file saved as: cleaned_medical_reports_analysis.csv
  Total records (test entries): 76
  Unique standardized test names found: 65

ANALYSIS COMPLETE!
A single cleaned CSV has been generated: 'cleaned_medical_reports_analysis.csv'
  It contains 76 test entries.

Do you want to download the 'cleaned_medical_reports_analysis.csv' file? (y/n): y


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Processing finished. The CSV file is saved in the Colab environment.
You can find it in the directory: /content

Script setup complete.
If the Gemini API is configured, you can run the `main()` function cell to start processing.
Instructions:
1. Ensure your Gemini API key is added to Colab secrets (View -> Secrets -> Add new secret 'GEMINI_API_KEY').
2. Upload your PDF medical reports to a folder in your Google Drive.
3. When prompted by `main()`, enter the full path to that Google Drive folder.
4. The script will process the PDFs and generate a single, cleaned CSV file named 'cleaned_medical_reports_analysis.csv'.
